In [1]:
import tensorflow as tf
from tensorflow.keras import layers, models
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# 경로지정
answer_path = '/content/drive/MyDrive/GDSC_AI/BARO/source/answer_jpg' # 동영상 프레임 단위로 짜른거
wrong_path = '/content/drive/MyDrive/GDSC_AI/BARO/source/wrong_jpg' # 동영상 프레임 단위로 짜른거

In [3]:
## 전처리

def load_and_preprocess_images(folder_path, label, img_size=(28, 28)):
    images = []
    labels = []

    for img_file in os.listdir(folder_path):
        img_path = os.path.join(folder_path, img_file)
        img = cv2.imread(img_path)
        img = cv2.resize(img, img_size)
        img = img / 255.0  # 정규화
        images.append(img)
        labels.append(label)

    return images, labels

In [4]:
# label 할당(1값: 정상, 0값: 거북이)

answer_images, answer_labels = load_and_preprocess_images(answer_path, label=1)
wrong_images, wrong_labels = load_and_preprocess_images(wrong_path, label=0)

In [5]:
images = np.array(answer_images + wrong_images)
labels = np.array(answer_labels + wrong_labels)

In [6]:
X_train, X_test, y_train, y_test = train_test_split(images, labels, test_size=0.2, random_state=42)

In [7]:
model = models.Sequential([
    layers.Conv2D(32, (3, 3), activation='relu', input_shape=(28, 28, 3)),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),
    layers.Conv2D(64, (3, 3), activation='relu'),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(1, activation='sigmoid')  # 이진 분류 위한 softmax에서 sigmoid로 변경
])

/usr/local/lib/python3.10/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [9]:
history = model.fit(X_train, y_train, epochs=10, validation_data=(X_test, y_test))

Epoch 1/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 2s 57ms/step - accuracy: 0.4324 - loss: 0.7054 - val_accuracy: 0.5172 - val_loss: 0.6726
Epoch 2/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.6285 - loss: 0.6678 - val_accuracy: 0.8966 - val_loss: 0.6257
Epoch 3/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.7668 - loss: 0.6290 - val_accuracy: 0.8966 - val_loss: 0.5430
Epoch 4/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 46ms/step - accuracy: 0.8369 - loss: 0.5832 - val_accuracy: 0.8966 - val_loss: 0.4510
Epoch 5/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step - accuracy: 0.8496 - loss: 0.5124 - val_accuracy: 0.8966 - val_loss: 0.3817
Epoch 6/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.8649 - loss: 0.3989 - val_accuracy: 0.9483 - val_loss: 0.2668
Epoch 7/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 0s 48ms/step - accuracy: 0.8977 - loss: 0.3460 - val_accuracy: 0.9483 - val_loss: 0.1975
Epoch 8/10
8/8 ━━━━━━━━━━━━━━━━━━━━ 1s 53ms/step - accuracy: 0.8930 - loss: 0.3019 - val_accuracy: 0.8966 - val_loss: 0.2635


In [13]:
test_loss, test_accuracy = model.evaluate(X_test, y_test, verbose=2)
print('Test accuracy: ', test_accuracy)

2/2 - 0s - 39ms/step - accuracy: 0.9483 - loss: 0.1441
Test accuracy:  0.9482758641242981


In [ ]:
model.save("CNN_model.h5")

In [ ]:
model_tfl = tf.keras.models.load_model('/content/CNN_model.h5') # 모델 .h5 파일 형태로 저장

In [ ]:
converter = tf.lite.TFLiteConverter.from_keras_model(model_tfl)
tflite_model = converter.convert() # h5 파일 .tflite 파일로 변환

Saved artifact at '/tmp/tmp6chf57xh'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 28, 28, 3), dtype=tf.float32, name='input_layer')
Output Type:
  TensorSpec(shape=(None, 2), dtype=tf.float32, name=None)
Captures:
  134981498079888: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134981498083584: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134981498176080: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134981498176256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134981498187168: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134981498189456: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134981291640256: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134981291643072: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134981291643776: TensorSpec(shape=(), dtype=tf.resource, name=None)
  134981291647296: TensorSpec(shape=(), dtype=tf.resource, name=None)


In [ ]:
with open('CNN_model.tflite', 'wb') as f:
  f.write(tflite_model) # tflite 모델 저장.